# Train Random Forest Model với Area/Width/Length
Notebook này sẽ train lại model Random Forest có bao gồm các thông số Area, Width và Length.

In [10]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

## 1. Load và Chuẩn bị Dữ liệu

In [11]:
# Load dữ liệu
data_path = '../web_app/dash/streamlit/data/cleaned_data.csv'
df = pd.read_csv(data_path)
print(f"Tổng số dòng: {len(df)}")
print(f"Các cột: {df.columns.tolist()}")
df.head()

Tổng số dòng: 81115
Các cột: ['District', 'Ward', 'House_type', 'Legal_documents', 'No_floor', 'No_bedroom', 'Length', 'Width', 'Price', 'Day_Of_Week', 'Month', 'Street', 'Area', 'Price_range']


,District,Ward,House_type,Legal_documents,No_floor,No_bedroom,Length,Width,Price,Day_Of_Week,Month,Street,Area,Price_range
0,CẦU GIẤY,NGHĨA ĐÔ,BYROAD,AVAILABLE,4,5,11.00,4.0,86.96,WEDNESDAY,8,HOÀNG QUỐC VIỆT,44.0,81-90
1,THANH XUÂN,KIM GIANG,STREET_HOUSE,OTHERS,5,3,10.00,4.0,116.22,WEDNESDAY,8,KIM GIANG,40.0,101-200
2,HAI BÀ TRƯNG,MINH KHAI,BYROAD,AVAILABLE,4,4,10.00,4.0,65.00,WEDNESDAY,8,MINH KHAI,40.0,61-70
3,TÂY HỒ,THỤY KHUÊ,BYROAD,AVAILABLE,5,6,12.75,4.0,100.00,WEDNESDAY,8,VÕNG THỊ,51.0,91-100
4,THANH XUÂN,KIM GIANG,BYROAD,OTHERS,5,4,9.00,4.0,86.11,WEDNESDAY,8,KIM GIANG,36.0,81-90


In [12]:
# Xóa các cột không cần thiết (GIỮ LẠI Area, Width, Length)
cols_to_drop = ['Price', 'Street']
df = df.drop(columns=cols_to_drop, errors='ignore')

# Thêm cột Region
def get_region(district):
    urban = ['CẦU GIẤY', 'THANH XUÂN', 'HAI BÀ TRƯNG', 'TÂY HỒ', 'ĐỐNG ĐA', 'HOÀNG MAI', 'HOÀN KIẾM', 'BA ĐÌNH']
    return 1 if district in urban else 0

df['Region'] = df['District'].apply(get_region)

print(f"Các cột sau khi xử lý: {df.columns.tolist()}")
df.head()

Các cột sau khi xử lý: ['District', 'Ward', 'House_type', 'Legal_documents', 'No_floor', 'No_bedroom', 'Length', 'Width', 'Day_Of_Week', 'Month', 'Area', 'Price_range', 'Region']


,District,Ward,House_type,Legal_documents,No_floor,No_bedroom,Length,Width,Day_Of_Week,Month,Area,Price_range,Region
0,CẦU GIẤY,NGHĨA ĐÔ,BYROAD,AVAILABLE,4,5,11.00,4.0,WEDNESDAY,8,44.0,81-90,1
1,THANH XUÂN,KIM GIANG,STREET_HOUSE,OTHERS,5,3,10.00,4.0,WEDNESDAY,8,40.0,101-200,1
2,HAI BÀ TRƯNG,MINH KHAI,BYROAD,AVAILABLE,4,4,10.00,4.0,WEDNESDAY,8,40.0,61-70,1
3,TÂY HỒ,THỤY KHUÊ,BYROAD,AVAILABLE,5,6,12.75,4.0,WEDNESDAY,8,51.0,91-100,1
4,THANH XUÂN,KIM GIANG,BYROAD,OTHERS,5,4,9.00,4.0,WEDNESDAY,8,36.0,81-90,1


## 2. Xác định Features và Target

In [13]:
# Tách X và y
X = df.drop('Price_range', axis=1)
y = df['Price_range']

print(f"Features: {X.columns.tolist()}")
print(f"Target classes: {y.unique()}")

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"\nTrain size: {len(X_train)}, Test size: {len(X_test)}")

Features: ['District', 'Ward', 'House_type', 'Legal_documents', 'No_floor', 'No_bedroom', 'Length', 'Width', 'Day_Of_Week', 'Month', 'Area', 'Region']
Target classes: ['81-90' '101-200' '61-70' '91-100' '1-60' '71-80' '201-300' '301-1000']

Train size: 56780, Test size: 24335


## 3. Tạo Pipeline với Preprocessing

In [14]:
# Xác định các loại cột
categorical_cols = ['District', 'Ward', 'House_type', 'Legal_documents', 'Day_Of_Week']
ordinal_cols = ['No_floor', 'No_bedroom']
numerical_cols = ['Month', 'Region', 'Area', 'Width', 'Length']  # THÊM Area, Width, Length

# Tạo Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('ord', OrdinalEncoder(), ordinal_cols)
    ],
    remainder='passthrough'  # Giữ nguyên các cột số (numerical_cols)
)

# Tạo Pipeline với Random Forest
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        random_state=42,
        n_jobs=-1
    ))
])

print("Pipeline đã được tạo!")

Pipeline đã được tạo!


## 4. Train Model

In [15]:
# Train
print("Đang train model...")
pipeline.fit(X_train, y_train)
print("Train hoàn tất!")

# Đánh giá
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nĐộ chính xác trên tập test: {accuracy:.2%}")

print("\nBáo cáo chi tiết:")
print(classification_report(y_test, y_pred))

Đang train model...
Train hoàn tất!

Độ chính xác trên tập test: 39.10%

Báo cáo chi tiết:
              precision    recall  f1-score   support

        1-60       0.55      0.58      0.56      2925
     101-200       0.38      0.92      0.53      7113
     201-300       0.62      0.01      0.01       782
    301-1000       0.67      0.08      0.14       392
       61-70       0.43      0.07      0.11      2447
       71-80       0.31      0.19      0.24      3541
       81-90       0.32      0.09      0.14      3631
      91-100       0.42      0.01      0.02      3504

    accuracy                           0.39     24335
   macro avg       0.46      0.24      0.22     24335
weighted avg       0.40      0.39      0.30     24335



## 5. Lưu Model

In [16]:
# Lưu model vào thư mục models
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, 'randomForest_with_area_pipeline.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(pipeline, f)

print(f"Model đã được lưu tại: {model_path}")
print(f"\nFeatures mà model sử dụng: {X.columns.tolist()}")

Model đã được lưu tại: ../models\randomForest_with_area_pipeline.pkl

Features mà model sử dụng: ['District', 'Ward', 'House_type', 'Legal_documents', 'No_floor', 'No_bedroom', 'Length', 'Width', 'Day_Of_Week', 'Month', 'Area', 'Region']


## 6. Test thử Model

In [17]:
# Load model và test
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

# Tạo dữ liệu test
test_input = pd.DataFrame([{
    'District': 'CẦU GIẤY',
    'Ward': 'NGHĨA ĐÔ',
    'House_type': 'TOWNHOUSE',
    'Legal_documents': 'AVAILABLE',
    'No_floor': '4',
    'No_bedroom': '3',
    'Day_Of_Week': 'MONDAY',
    'Month': 8,
    'Region': 1,
    'Area': 50.0,
    'Width': 5.0,
    'Length': 10.0
}])

prediction = loaded_model.predict(test_input)
proba = loaded_model.predict_proba(test_input)

class_names = ['1-60', '61-70', '71-80', '81-90', '91-100', '101-200', '201-300', '301-1000']
print(f"Dự đoán: {class_names[prediction[0]]} triệu VNĐ")
print(f"\nXác suất các khoảng giá:")
for i, name in enumerate(class_names):
    print(f"  {name}: {proba[0][i]:.2%}")

TypeError: list indices must be integers or slices, not str

In [ ]:
# Test với Area khác để xem model có phản hồi không
test_input2 = test_input.copy()
test_input2['Area'] = 200.0  # Tăng diện tích lên
test_input2['Width'] = 10.0
test_input2['Length'] = 20.0

prediction2 = loaded_model.predict(test_input2)
print(f"\nVới Area=200, Width=10, Length=20:")
print(f"Dự đoán: {class_names[prediction2[0]]} triệu VNĐ")


Với Area=200, Width=10, Length=20:


TypeError: list indices must be integers or slices, not str